# Agentic CFO Copilot: AIML Engine Demonstration

This notebook provides a complete, end-to-end demonstration of the AIML engine. We will walk through each core module, from ingesting a raw CSV file to generating a final, dashboard-ready JSON output.

### Step 0: Setup and Imports

First, let's import all the necessary modules from our `aiml_engine` and other libraries. Make sure you have installed all dependencies from `requirements.txt`.

In [1]:
import pandas as pd
import sys
import os
import json
import plotly.graph_objects as go

# Add the project root to the Python path to allow for module imports
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from aiml_engine.core.data_ingestion import DataIngestion
from aiml_engine.core.data_validation import DataValidationQualityAssuranceEngine
from aiml_engine.core.feature_engineering import KPIAutoExtractionDynamicFeatureEngineering
from aiml_engine.core.forecasting import ForecastingModule
from aiml_engine.core.anomaly_detection import AnomalyDetectionModule
from aiml_engine.core.correlation import CrossMetricCorrelationTrendMiningEngine
from aiml_engine.core.simulation import ScenarioSimulationEngine
from aiml_engine.core.dashboard import BusinessDashboardOutputLayer
from aiml_engine.utils.helpers import serialize_to_json

# Define the path to our sample data
DATA_FILE_PATH = '../aiml_engine/data/sample_financial_data.csv'

/Users/swayamsahoo/Projects/praxify-CFO/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Step 1: Data Ingestion and Normalization

We start by feeding a raw CSV file to the `DataIngestion` module. This module will automatically detect the column meanings (e.g., mapping `Sales Revenue` to `revenue`) and create a standardized DataFrame.

In [2]:
ingestion_module = DataIngestion()
normalized_df, header_mappings = ingestion_module.ingest_and_normalize(DATA_FILE_PATH)

print("--- Header Mappings ---")
print(json.dumps(header_mappings, indent=2))
print("\n--- Normalized DataFrame Head ---")
display(normalized_df.head())

/Users/swayamsahoo/Projects/praxify-CFO/aiml_engine/core/data_ingestion.py:45: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = col_doc.similarity(synonym_doc)
/Users/swayamsahoo/Projects/praxify-CFO/aiml_engine/core/data_ingestion.py:45: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use 

--- Header Mappings ---
{
  "Transaction Date": "date",
  "Sales Revenue": "revenue",
  "Operating Costs": "expenses",
  "Marketing Spend": "cashflow",
  "Region": "region",
  "Department": "department",
  "Net Profit": "profit",
  "Accounts Receivable": "ar",
  "Accounts Payable": "ap",
  "Total Assets": "assets",
  "Total Liabilities": "liabilities"
}

--- Normalized DataFrame Head ---


/Users/swayamsahoo/Projects/praxify-CFO/aiml_engine/core/data_ingestion.py:45: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = col_doc.similarity(synonym_doc)


,date,revenue,expenses,cashflow,region,department,profit,Cash Flow from Operations,ar,ap,assets,liabilities
0,2023-01-31,120500,75300.0,15000.0,NAM,Sales,30200,25000.0,45000,22000,550000,280000
1,2023-02-28,115000,72000.0,14500.0,NAM,Marketing,28500,23000.0,43000,21000,552000,281000
2,2023-03-31,135000,80000.0,16000.0,APAC,Sales,39000,32000.0,48000,24000,560000,285000
3,2023-04-30,145000,85000.0,17000.0,APAC,Operations,43000,35000.0,52000,26000,565000,288000
4,2023-05-31,152000,88000.0,18000.0,EMEA,Sales,46000,NaN,55000,27000,570000,290000


### Step 2: Data Validation and Quality Assurance

Next, we pass the normalized data through the validation engine. It will handle missing values, coerce data types, and produce a `validation_report` and a `corrections_log` detailing every change made.

In [3]:
validation_module = DataValidationQualityAssuranceEngine()
validated_df, validation_report, corrections_log = validation_module.run_pipeline(normalized_df, header_mappings)

print("--- Validation Report ---")
print(json.dumps(validation_report, indent=2))
print("\n--- Corrections Log (Sample) ---")
print(json.dumps(corrections_log[:3], indent=2))

--- Validation Report ---
{
  "dataset_id": "ds_1760196905",
  "original_shape": [
    23,
    12
  ],
  "cleaned_shape": [
    23,
    12
  ],
  "missing_values_imputed": true,
  "imputed_columns_summary": {
    "cashflow": 1,
    "expenses": 1
  },
  "header_mappings": {
    "Transaction Date": "date",
    "Sales Revenue": "revenue",
    "Operating Costs": "expenses",
    "Marketing Spend": "cashflow",
    "Region": "region",
    "Department": "department",
    "Net Profit": "profit",
    "Accounts Receivable": "ar",
    "Accounts Payable": "ap",
    "Total Assets": "assets",
    "Total Liabilities": "liabilities"
  }
}

--- Corrections Log (Sample) ---
[
  {
    "row_id": 9,
    "column": "expenses",
    "original": "N/A",
    "correction": 91000.0,
    "method": "median_impute",
    "timestamp": "2025-10-11T15:35:05.201697Z"
  },
  {
    "row_id": 7,
    "column": "cashflow",
    "original": "N/A",
    "correction": 21500.0,
    "method": "median_impute",
    "timestamp": "2025-10-

### Step 3: KPI Auto-Extraction & Feature Engineering

Now, we derive important financial KPIs and features like `profit_margin` and MoM growth rates. The `feature_schema` documents the origin and transformation of each new feature.

In [4]:
feature_module = KPIAutoExtractionDynamicFeatureEngineering()
featured_df, feature_schema = feature_module.extract_and_derive_features(validated_df)

print("--- Feature Schema ---")
print(json.dumps(feature_schema, indent=2))
print("\n--- DataFrame with New Features (Head) ---")
display(featured_df[['date', 'revenue', 'expenses', 'profit', 'profit_margin', 'revenue_mom_growth']].head())

--- Feature Schema ---
[
  {
    "feature": "profit_margin",
    "type": "float64",
    "source": [
      "revenue",
      "expenses"
    ],
    "transformation": "(revenue - expenses) / revenue"
  },
  {
    "feature": "debt_to_asset_ratio",
    "type": "float64",
    "source": [
      "liabilities",
      "assets"
    ],
    "transformation": "liabilities / assets"
  },
  {
    "feature": "dso",
    "type": "float64",
    "source": [
      "ar",
      "revenue",
      "date"
    ],
    "transformation": "(accounts_receivable / revenue) * days_in_period"
  },
  {
    "feature": "revenue_mom_growth",
    "type": "float64",
    "source": [
      "revenue"
    ],
    "transformation": "pandas.pct_change()"
  },
  {
    "feature": "profit_mom_growth",
    "type": "float64",
    "source": [
      "profit"
    ],
    "transformation": "pandas.pct_change()"
  },
  {
    "feature": "expenses_mom_growth",
    "type": "float64",
    "source": [
      "expenses"
    ],
    "transformation": "pan

,date,revenue,expenses,profit,profit_margin,revenue_mom_growth
0,2023-01-31,120500,75300.0,30200,0.375104,0.000000
1,2023-02-28,115000,72000.0,28500,0.373913,-4.564315
2,2023-03-31,135000,80000.0,39000,0.407407,17.391304
3,2023-04-30,145000,85000.0,43000,0.413793,7.407407
4,2023-05-31,152000,88000.0,46000,0.421053,4.827586


### Step 4: Predictive Forecasting

The forecasting module automatically selects the best model (between AutoARIMA and Prophet) and generates a 3-month forecast for the specified metric, including confidence intervals.

In [5]:
forecasting_module = ForecastingModule(metric='revenue', date_col='date')
forecast, model_health = forecasting_module.generate_forecast(featured_df)

print("--- Model Health Report ---")
print(json.dumps(model_health, indent=2))
print("\n--- Forecast Results ---")
print(json.dumps(forecast, indent=2))

21:05:12 - cmdstanpy - INFO - Chain [1] start processing


21:05:22 - cmdstanpy - INFO - Chain [1] done processing
21:05:22 - cmdstanpy - INFO - Chain [1] start processing
21:05:34 - cmdstanpy - INFO - Chain [1] done processing


--- Model Health Report ---
{
  "model_id": "model_prophet_1760196934",
  "best_model_selected": "Prophet",
  "backtesting_rmse": {
    "AutoARIMA": "N/A",
    "Prophet": 60743.83267240184
  },
  "forecast_metric": "revenue",
  "status": "Success"
}

--- Forecast Results ---
[
  {
    "date": "2025-01-01",
    "predicted": 74250.69104991574,
    "lower": 73876.96768909512,
    "upper": 74583.56318665059
  },
  {
    "date": "2025-02-01",
    "predicted": 59672.22031548683,
    "lower": 58421.349929879434,
    "upper": 60874.34981333327
  },
  {
    "date": "2025-03-01",
    "predicted": 135512.58040653213,
    "lower": 133057.56160412487,
    "upper": 137943.28703137254
  }
]


#### Visualizing the Forecast

Let's plot the historical data along with the forecast to visualize the results.

In [6]:
if forecast:
    forecast_df = pd.DataFrame(forecast)
    forecast_df['date'] = pd.to_datetime(forecast_df['date'])

    fig = go.Figure()
    # Historical Data
    fig.add_trace(go.Scatter(x=featured_df['date'], y=featured_df['revenue'], mode='lines', name='Historical Revenue'))
    # Forecast Data
    fig.add_trace(go.Scatter(x=forecast_df['date'], y=forecast_df['predicted'], mode='lines', name='Forecasted Revenue', line=dict(dash='dash')))
    # Confidence Interval
    fig.add_trace(go.Scatter(x=forecast_df['date'], y=forecast_df['upper'], fill=None, mode='lines', line_color='rgba(0,0,0,0)', name='Upper CI'))
    fig.add_trace(go.Scatter(x=forecast_df['date'], y=forecast_df['lower'], fill='tonexty', mode='lines', line_color='rgba(0,0,0,0)', name='Lower CI'))

    fig.update_layout(title='Revenue Forecast', xaxis_title='Date', yaxis_title='Revenue')
    fig.show()
else:
    print("No forecast generated. Skipping visualization.")

### Step 5: Anomaly Detection

The `AnomalyDetectionModule` scans the data for significant outliers and reports them with severity and a reason.

In [7]:
anomaly_module = AnomalyDetectionModule()
anomalies = anomaly_module.detect_anomalies(featured_df, metric='revenue')

print("--- Detected Anomalies ---")
print(json.dumps(anomalies, indent=2))

--- Detected Anomalies ---
[]


### Step 6: Cross-Metric Correlation & Trend Mining

Here, we uncover hidden relationships between different financial metrics.

In [8]:
correlation_module = CrossMetricCorrelationTrendMiningEngine()
correlation_report = correlation_module.generate_correlation_report(featured_df)

print("--- Correlation Report ---")
print(json.dumps(correlation_report, indent=2))

--- Correlation Report ---
[
  {
    "metric_a": "revenue",
    "metric_b": "profit",
    "correlation": 0.9934990172727183
  },
  {
    "metric_a": "revenue",
    "metric_b": "expenses",
    "correlation": 0.9879112574939233
  },
  {
    "metric_a": "expenses",
    "metric_b": "profit",
    "correlation": 0.9816592430411692
  },
  {
    "metric_a": "revenue",
    "metric_b": "cashflow",
    "correlation": 0.9807589961685879
  },
  {
    "metric_a": "expenses",
    "metric_b": "cashflow",
    "correlation": 0.9723064979193916
  },
  {
    "metric_a": "profit",
    "metric_b": "cashflow",
    "correlation": 0.9571463020349771
  }
]


/Users/swayamsahoo/Projects/praxify-CFO/venv/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:1545: FutureWarning:

verbose is deprecated since functions should not print results

/Users/swayamsahoo/Projects/praxify-CFO/venv/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:1545: FutureWarning:

verbose is deprecated since functions should not print results

/Users/swayamsahoo/Projects/praxify-CFO/venv/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:1545: FutureWarning:

verbose is deprecated since functions should not print results

/Users/swayamsahoo/Projects/praxify-CFO/venv/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:1545: FutureWarning:

verbose is deprecated since functions should not print results

/Users/swayamsahoo/Projects/praxify-CFO/venv/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:1545: FutureWarning:

verbose is deprecated since functions should not print results

/Users/swayamsahoo/Projects/praxify-CFO/venv/lib/python3.9/s

### Step 7: Scenario Simulation (What-If Engine)

Let's simulate the impact of a 15% increase in expenses on profit and cashflow.

In [9]:
simulation_module = ScenarioSimulationEngine()
simulation_results = simulation_module.simulate_scenario(df=featured_df, parameter='expenses', change_pct=15.0)

print("--- Simulation Results ---")
print(serialize_to_json(simulation_results))

--- Simulation Results ---
{
    "scenario": {
        "parameter_changed": "expenses",
        "change_percentage": 15.0
    },
    "baseline": {
        "total_profit": 1223200,
        "total_cashflow": 522500.0
    },
    "simulation_results": {
        "total_profit": 1428605.0000000002,
        "total_cashflow": 610240.4451438849
    },
    "impact": {
        "profit_impact_absolute": 205405.00000000023,
        "profit_impact_percentage": 16.792429692609566,
        "cashflow_impact_absolute": 87740.44514388486,
        "cashflow_impact_percentage": 16.79242969260954
    },
    "summary_text": "A 15.0% change in 'expenses' is projected to change total profit by 205,405.00 (16.79%) and total cashflow by 87,740.45 (16.79%)."
}


### Step 8: Final Dashboard Generation

Finally, we bring everything together into a single, cohesive JSON object ready to be consumed by a frontend dashboard. We will generate it in both 'Finance Guardian' and 'Financial Storyteller' modes.

In [10]:
dashboard_module = BusinessDashboardOutputLayer()

# --- Finance Guardian Mode --- #
guardian_output = dashboard_module.generate_dashboard(
    featured_df=featured_df,
    forecast=forecast,
    anomalies=anomalies,
    mode="finance_guardian",
    correlation_report=correlation_report,
    simulation_results=simulation_results
)

print("="*50)
print("          FINANCE GUARDIAN OUTPUT         ")
print("="*50)
print(serialize_to_json(guardian_output))

# --- Financial Storyteller Mode --- #
storyteller_output = dashboard_module.generate_dashboard(
    featured_df=featured_df,
    forecast=forecast,
    anomalies=anomalies,
    mode="financial_storyteller",
)

print("\n" + "="*50)
print("        FINANCIAL STORYTELLER OUTPUT      ")
print("="*50)
print(serialize_to_json(storyteller_output))

          FINANCE GUARDIAN OUTPUT         
{
    "dashboard_mode": "finance_guardian",
    "metadata": {
        "generated_at": "2025-10-11T15:36:15.154595Z",
        "data_start_date": "2023-01-31",
        "data_end_date": "2024-12-31"
    },
    "kpis": {
        "total_revenue": 3932500,
        "total_expenses": 2177300.0,
        "profit_margin": 0.31104895104895103,
        "cashflow": 522500.0,
        "growth_rate": 4.551365409622887,
        "forecast_accuracy": 95.8,
        "financial_health_score": 48.36,
        "dso": 10.74989785013866
    },
    "forecast_chart": [
        {
            "date": "2025-01-01",
            "predicted": 74250.69104991574,
            "lower": 73876.96768909512,
            "upper": 74583.56318665059
        },
        {
            "date": "2025-02-01",
            "predicted": 59672.22031548683,
            "lower": 58421.349929879434,
            "upper": 60874.34981333327
        },
        {
            "date": "2025-03-01",
          